In [54]:
import h5py
import pandas as pd
import numpy as np
from decimal import Decimal

In [62]:
def get_speed(vel):
    assert(len(vel) == 3)
    return (Decimal(vel[0]) ** Decimal(2) + Decimal(vel[1]) ** Decimal(2) + Decimal(vel[2]) ** Decimal(2)) ** Decimal(0.5)

def angle(v1, v2):
    v1 = [Decimal(v1[0].item()), Decimal(v1[1].item()), Decimal(v1[2].item())]
    v2 = [Decimal(v2[0].item()), Decimal(v2[1].item()), Decimal(v2[2].item())]
    # [0, np.pi]
    num = np.dot(v1, v2)
    denom = (get_speed(v1) * get_speed(v2))
    return np.arccos(float(num / denom)) 

def dist(p1, p2):
    return get_speed(np.array(p1) - np.array(p2))

In [63]:
data_path = "/data1/jessica/data/toyota/once_w_lanes_compressed_raw_small_multitask_0-13.hfd5"
paths = [data_path]
people_seqs = []
subsample = 2
with h5py.File(data_path, "r") as f:
    for i, seq_key in enumerate(list(f.keys())):
        iter_dict = {}
        keys_ = f[seq_key].keys()
        for key in keys_:
            if key == "angle": continue
            ds_obj = f[seq_key][key][()]#[0::subsample]
            if key == "pos":
                iter_dict[key] = ds_obj
                new_angles = [0]
                for i in range(1, len(ds_obj)):
                    prev = ds_obj[i-1]
                    curr = ds_obj[i]
                    an = angle(prev, curr)*(180/np.pi)
                    new_angles.append(an)
                ds_obj = new_angles
                iter_dict["angle"] = ds_obj
                continue
            iter_dict[key] = ds_obj
        people_seqs.append(iter_dict)


In [64]:
people_seqs[0]['pos']

KeyError: 'pos'

In [42]:
people_seqs[0].keys()


dict_keys(['angle', 'distance', 'image_array', 'orig_times', 'pos', 'segm_masks', 'speed'])

In [59]:
def get_speed(vel):
    return (Decimal(vel[0]) ** Decimal(2) + Decimal(vel[1]) ** Decimal(2)) ** Decimal(0.5)
def angle(v1, v2):
    v1 = [Decimal(v1[0]), Decimal(v1[1])]
    v2 = [Decimal(v2[0]), Decimal(v2[1])]
    # [0, np.pi]
    num = np.dot(v1, v2)
    denom = (get_speed(v1) * get_speed(v2))
    return np.arccos(float(num / denom)) 

angle([,1], [1,2])*(180/np.pi)

18.434948822922017